<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

How to use the Radiant MLHub API to browse and download the BigEarthNet dataset
=====

This Jupyter notebook, which you may copy and adapt for any use, shows basic examples of how to use the API to download labels and source imagery for the BigEarthNet dataset. Full documentation for the API is available at [docs.mlhub.earth](docs.mlhub.earth).

We'll show you how to set up your authorization, see the list of available collections and datasets, and retrieve the items (the data contained within them) from those collections. 

Each item in our collection is explained in json format compliant with [STAC](https://stacspec.org/) [label extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/label) definition.

Authentication
-----

Access to the Radiant MLHub API requires an access token. To get your access token, go to [dashboard.mlhub.earth](https://dashboard.mlhub.earth). If you have not used Radiant MLHub before, you will need to sign up and create a new account. Otherwise, sign in. Under **Usage**, you'll see your access token, which you will need. *Do not share* your access token with others: your usage may be limited and sharing your access token is a security risk.

Copy the access token, and paste it in the box bellow. This header block will work for all API calls.

Click **Run** or press `SHIFT` + `ENTER` before moving on to run this first piece of code.

In [1]:
# only the requests module is required to access the API
import requests

# copy your access token from dashboard.mlhub.earth and paste it in the following
ACCESS_TOKEN = 'PASTE_YOUR_ACCESS_TOKEN_HERE'

# these headers will be used in each request
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}',
    'Accept':'application/json'
}

Search for data collections
-----

To see what training data is available, you will want to see the collections available through the API.

A collection represents the top-most data level. Typically this means the data comes from the same source for the same geography. It might include different years or sub-geographies.

To find data with specific parameters, see the [API documentation](http://docs.mlhub.earth/?python#the-feature-collections-in-the-dataset).

To see the list, simply run the following cell. The returned list shows the collection id values, collection license, and data source citation (if available).

In [2]:
# get list of all collections
r = requests.get('https://api.radiant.earth/mlhub/v1/collections', headers=headers)
h = r.json()
collections = h['collections']

# print the list of collections 
for c in collections:
    print(f'ID:       {c["id"]}\nLicense:  {c.get("license", "N/A")}\nCitation: {c.get("sci:citation", "N/A")}\n')

ID:       ref_african_crops_uganda_01
License:  CC-BY-SA-4.0
Citation: Bocquet, C., Dalberg Data Insights. (2019) Dalberg Data Insights Uganda Crop Classification, Version 1. [Indicate subset used]. Radiant ML Hub. [Date Accessed]

ID:       microsoft_chesapeake_nlcd
License:  CDLA-permissive-1.0
Citation: Robinson C, Hou L, Malkin K, Soobitsky R, Czawlytko J, Dilkina B, Jojic N. Large Scale High-Resolution Land Cover Mapping with Multi-Resolution Data. Proceedings of the 2019 Conference on Computer Vision and Pattern Recognition (CVPR 2019).

ID:       ref_african_crops_tanzania_01
License:  CC-BY-SA-4.0
Citation: Great African Food Company. (2019) Great African Food Company Tanzania Ground Reference Crop Type Dataset, Version 1. [Indicate subset used]. Radiant ML Hub. [Date Accessed]

ID:       ref_african_crops_kenya_02_source
License:  CC-BY-SA-4.0
Citation: Radiant Earth Foundation (2020) CV4A Competition Kenya Crop Type Dataset, Version 1. [Indicate subset used]. Radiant ML Hub. 

Setting the query properties
----

The BigEarthNet dataset is split into two collections, one which contains the labels and one which contains the source imagery. Labels link to their respective source imagery items so we will set our collection ID to `bigearthnet_v1_labels`.

You can  limit what data you get in the response using the optional parameters:
* **Limit** limits how many items will be returned, with a minimum of 1 and maximum of 10000.

We will limit the amount of results returned per page to 100 items.

In [3]:
collectionId = 'bigearthnet_v1_labels'
limit = 100

Downloading Items
----

The next cell contains the functions which page through the results and download the labels and source imagery.

In [4]:
import boto3 # Required to download assets hosted on S3
import os
from urllib.parse import urlparse

s3 = boto3.client('s3')

def download_s3(uri, path):
    parsed = urlparse(uri)
    bucket = parsed.netloc
    key = parsed.path[1:]
    s3.download_file(bucket, key, os.path.join(path, key.split('/')[-1]))
    print(f'Downloaded s3://{bucket}/{key}')
    
def download_http(uri, path):
    parsed = urlparse(uri)
    r = requests.get(uri)
    f = open(os.path.join(path, parsed.path[1:]), 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk:
            f.write(chunk)
    f.close()
    print(f'Downloaded {uri}')

def get_download_uri(uri):
    r = requests.get(uri, allow_redirects=False)
    return r.headers['Location']

def download(href, path):
    download_uri = get_download_uri(href)
    parsed = urlparse(download_uri)
    
    if parsed.scheme in ['s3']:
        download_s3(download_uri, path)
    elif parsed.scheme in ['http', 'https']:
        download_http(download_uri, path)

def download_source_and_labels(item):
    labels = item.get('assets').get('labels')
    links = item.get('links')
    
    # Make the directory to download the files to
    path = f'bigearthnet/{item["id"]}/'
    if not os.path.exists(path):
        os.makedirs(path)
    
    # Download the labels
    download(labels['href'], path)
    
    #Download the source imagery
    for link in links:
        if link['rel'] != 'source':
            continue
        
        r = requests.get(link['href'], headers=headers)
        for key, asset in r.json()['assets'].items():
            download(asset['href'], path)
            
def get_items(uri, classes=None, cloud_and_shadow=None, seasonal_snow=None, max_items_downloaded=None, items_downloaded=0):
    r = requests.get(uri, headers=headers)
    collection = r.json()
    for feature in collection.get('features', []):
        # Check if the item has one of the label classes we're interested in
        matches_class = True
        if classes is not None:
            matches_class = False
            for label_class in feature['properties'].get('labels', []):
                if label_class in classes:
                    matches_class = True
                    break
        
        # Check if the item matches the cloud and shadows filter we specify
        matches_clouds = True
        if cloud_and_shadow is not None:
            matches_clouds = feature['properties'].get('cloud_and_shadow', False) == cloud_and_shadow
            
        
        # Check if the item matches the seasonal snow filter we specify
        matches_snow = True
        if seasonal_snow is not None:
            matches_snow = feature['properties'].get('seasonal_snow', False) == seasonal_snow
            
        # If the item does not match all of the criteria we specify, skip it
        if not matches_class or not matches_clouds or not matches_snow:
            continue
            
        # Download the label and source imagery for the item
        download_source_and_labels(feature)
        
        # Stop downloaded items if we reached the maximum we specify
        items_downloaded += 1
        if max_items_downloaded is not None and items_downloaded >= max_items_downloaded:
            return
        
    # Get the next page if results, if available
    for link in collection['links']:
        if link['rel'] == 'next' and link['href'] is not None:
            get_items(link['href'], classes=classes, cloud_and_shadow=cloud_and_shadow, seasonal_snow=seasonal_snow, max_items_downloaded=max_items_downloaded, items_downloaded=items_downloaded)

Listing total number of tiles
----

To find the total number of tiles in the dataset we will query the collection for an item and read the "numberMatched" property which contains the total number of tiles in the collection.

In [5]:
r = requests.get(f'https://api.radiant.earth/mlhub/v1/collections/{collectionId}/items?limit=1', headers=headers)
print(f'Total Number of Tiles: {r.json()["numberMatched"]}')

Total Number of Tiles: 590326


Downloading all labels
----

This next cell will download all labels and source imagery contained in the BigEarthnet dataset. For demonstration purposes in this notebook, we limit the number of items downloaded to 1. You can remove the `max_items_downloaded` argument and the function will download all 590,326 labels and source imagery.

In [6]:
get_items(f'https://api.radiant.earth/mlhub/v1/collections/{collectionId}/items?limit={limit}', max_items_downloaded=1)

Downloaded s3://radiant-mlhub/bigearthnet/labels/S2A_MSIL2A_20180529T115401_33_74.geojson
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_33_74/S2A_MSIL2A_20180529T115401_33_74_B01.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_33_74/S2A_MSIL2A_20180529T115401_33_74_B02.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_33_74/S2A_MSIL2A_20180529T115401_33_74_B03.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_33_74/S2A_MSIL2A_20180529T115401_33_74_B04.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_33_74/S2A_MSIL2A_20180529T115401_33_74_B05.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_33_74/S2A_MSIL2A_20180529T115401_33_74_B06.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_33_74/S2A_MSIL2A_20180529T115401_33_74_B07.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_33_74/S2A_MSIL2A_20180529T115

Filtering downloads based of labels
----

A likely scenario is you only want to download tiles which contain certain land cover classes or tiles which are not cloudy. First you'll need to know which land cover classes are contained in the dataset so you know which ones to filter on. The next cell will query the API and return the possible values for each label.

In [7]:
r = requests.get(f'https://api.radiant.earth/mlhub/v1/collections/{collectionId}/items?limit=1', headers=headers)
label_classes = r.json()['features'][0]['properties']['label:classes']
for label_class in label_classes:
    print(f'\nClasses for {label_class["name"]}')
    for c in sorted(label_class['classes']):
        print(f'- {c}')


Classes for labels
- Agro-forestry areas
- Airports
- Annual crops associated with permanent crops
- Bare rock
- Beaches, dunes, sands
- Broad-leaved forest
- Burnt areas
- Coastal lagoons
- Complex cultivation patterns
- Coniferous forest
- Construction sites
- Continuous urban fabric
- Discontinuous urban fabric
- Dump sites
- Estuaries
- Fruit trees and berry plantations
- Green urban areas
- Industrial or commercial units
- Inland marshes
- Intertidal flats
- Land principally occupied by agriculture, with significant areas of natural vegetation
- Mineral extraction sites
- Mixed forest
- Moors and heathland
- Natural grassland
- Non-irrigated arable land
- Olive groves
- Pastures
- Peatbogs
- Permanently irrigated land
- Port areas
- Rice fields
- Road and rail networks and associated land
- Salines
- Salt marshes
- Sclerophyllous vegetation
- Sea and ocean
- Sparsely vegetated areas
- Sport and leisure facilities
- Transitional woodland/shrub
- Vineyards
- Water bodies
- Water co

Filtering downloads on label classes, cloud and shadows, and seasonal snow
----

The labels in BigEarthNet have three properties.
1) An array of land cover type classes contained in the tile
2) Whether the tile contains cloud and cloud shadows
3) Whether the tile has seasonal snow

We can filter our download based off one or more of the properties.

In this next cell we will download the first 5 tiles which contain either the `Coniferous forest` or `Rice fields` classes, do not contain clouds and cloud shadows, and do not contain seasonal snow.

In [ ]:
get_items(
    f'https://api.radiant.earth/mlhub/v1/collections/{collectionId}/items?limit={limit}',
    classes=['Coniferous forest', 'Rice fields'],
    cloud_and_shadow=False,
    seasonal_snow=False,
    max_items_downloaded=5)

Downloaded s3://radiant-mlhub/bigearthnet/labels/S2A_MSIL2A_20180529T115401_65_71.geojson
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_65_71/S2A_MSIL2A_20180529T115401_65_71_B01.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_65_71/S2A_MSIL2A_20180529T115401_65_71_B02.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_65_71/S2A_MSIL2A_20180529T115401_65_71_B03.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_65_71/S2A_MSIL2A_20180529T115401_65_71_B04.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_65_71/S2A_MSIL2A_20180529T115401_65_71_B05.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_65_71/S2A_MSIL2A_20180529T115401_65_71_B06.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_65_71/S2A_MSIL2A_20180529T115401_65_71_B07.tif
Downloaded s3://radiant-mlhub/bigearthnet/S2A_MSIL2A_20180529T115401_65_71/S2A_MSIL2A_20180529T115